<a href="https://colab.research.google.com/github/mtrx-fin/Paper24/blob/main/customer_list_for_meta_audience_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Customer List Preparation for Meta Custom Audiences

This notebook:

1. Loads customer data from BigQuery and Google Sheets.
2. Cleans and normalizes the data (names, countries, values).
3. Merges datasets on `user_id`.
4. Prepares two export tables for Meta:
   - `meta_order_value.csv` — value-based audience on total order value.
   - `meta_rfm_score.csv` — value-based audience on RFM score.


## 1. Install and import libraries

In [ ]:
!pip install --quiet gspread gspread_dataframe google-cloud-bigquery pandas_gbq

import numpy as np
import pandas as pd

from google.colab import auth
import gspread
from gspread_dataframe import get_as_dataframe
from google.cloud import bigquery
from google.auth import default


## 2. Authenticate and basic configuration

We authenticate to Google APIs (BigQuery and Google Sheets) and define basic
configuration parameters (project ID, dataset, sheet ID, etc.).


In [ ]:
# Authenticate the Colab session for all Google APIs
auth.authenticate_user()

# BigQuery configuration
PROJECT_ID = "mtrx-analytics-bigquery-4e26"
BQ_USERS_TABLE = "mtrx-analytics-bigquery-4e26.analytics.users"  # full table name

# Google Sheets configuration
SPREADSHEET_ID = "1VsCFEXV4-RqolOkf7eAQTe78QmgyO5XgG33zXKFtZic"
SEGMENTATION_SHEET_NAME = "abcd segmentation"  # tab name in the spreadsheet


## 3. Load data from data sources

### 3.1 Segmentation data from Google Sheets

We read the customer segmentation table from Google Sheets using `gspread`.
The first row is treated as a header.


In [ ]:
def load_segmentation_sheet(spreadsheet_id: str, worksheet_name: str) -> pd.DataFrame:
    """Load segmentation data from Google Sheets into a pandas DataFrame."""

    creds, _ = default()
    gc = gspread.authorize(creds)
    sh = gc.open_by_key(spreadsheet_id)
    ws = sh.worksheet(worksheet_name)

    df = get_as_dataframe(
        ws,
        evaluate_formulas=True,
        header=0,  # first row contains headers
    )

    # Drop completely empty rows
    df = df.dropna(how="all").reset_index(drop=True)
    return df


df_seg_raw = load_segmentation_sheet(SPREADSHEET_ID, SEGMENTATION_SHEET_NAME)
df_seg_raw.head()


,Customer ID,Category,Email,Country,First Name,Last Name,Orders qty,Tips qty,Pages qty,Total Value,...,Avg User Revenue Per Category,LTV,Recency value,Frequency value,Monetary value,Recency score,Frequency score,Monetary score,Total Score,RFM Category
0,655f70deaa4b8e99744e3fe5,A,michelben71@gmail.com,Unknown,Unknown,Unknown,379,3,811.7,21579.48,...,1404.97,14315.18,1.1,196,11371.93,4,4,4,12,R4F4M4
1,644d5e4c9075a80e9e443028,A,marvinvillaflores22@yahoo.com,United States,Unknown,Unknown,183,3,727.2,14115.05,...,1404.97,9186.55,0.1,43,4185.98,4,4,4,12,R4F4M4
2,67119bd2c77d122a9d36fae2,A,vjgbq29kwk@privaterelay.appleid.com,United States,Unknown,Unknown,70,11,393.0,13095.82,...,1404.97,8605.13,25.3,61,11294.08,4,4,4,12,R4F4M4
3,6440fc598e5dd70cf4ce16b5,A,ddj0810@outlook.com,United States,Unknown,Unknown,248,10,854.7,12926.61,...,1404.97,7928.66,44.3,19,2204.22,3,4,4,11,R3F4M4
4,66f6bb03567e4ad01ff31e3e,A,bat764@yahoo.com,United States,Unknown,Unknown,61,0,396.8,12568.69,...,1404.97,8605.81,28.3,59,12284.28,4,4,4,12,R4F4M4


### 3.2 Customer data from BigQuery

We query the `users` table from BigQuery and keep only the fields
required for joining and building Meta audiences.


In [ ]:
def load_bq_users(project_id: str, table_fqn: str) -> pd.DataFrame:
    """Load customer data from BigQuery into a pandas DataFrame."""

    client = bigquery.Client(project=project_id)

    query = f"""
        SELECT
          id AS user_id,
          createdAt,
          lastActivityAt,
          firstName,
          lastName,
          city AS bgCity,
          country AS bgCountry,
          email AS bgEmail,
          emailConfirmed,
          statistic
        FROM `{table_fqn}`
        WHERE role = 'Customer'
    """

    df = client.query(query).to_dataframe()
    return df


df_bq_raw = load_bq_users(PROJECT_ID, BQ_USERS_TABLE)
df_bq_raw.head()


,user_id,createdAt,lastActivityAt,firstName,lastName,bgCity,bgCountry,bgEmail,emailConfirmed,statistic
0,63e3d7039d9c8739fca172aa,2023-02-08 17:08:19.930000+00:00,2023-02-24 07:08:53.636000+00:00,None,None,Rivne,Ukraine,solonska.a@gmail.com,false,"{""abandoned"":1,""bidding"":0,""canceled"":0,""close..."
1,63e4bab29d9c8739fca172ab,2023-02-09 09:19:46.188000+00:00,2023-02-21 15:35:04.719000+00:00,None,None,Rivne,Ukraine,olya9k@gmail.com,false,"{""abandoned"":2,""bidding"":0,""canceled"":0,""close..."
2,63e4bcbf9d9c8739fca172ac,2023-02-09 09:28:31.910000+00:00,2023-11-17 11:13:52.138000+00:00,None,None,Kyiv,Ukraine,danielsoloshenko@gmail.com,true,"{""abandoned"":4,""bidding"":0,""canceled"":0,""close..."
3,63e4bcce9d9c8739fca172ad,2023-02-09 09:28:46.138000+00:00,2023-08-26 12:44:26.203000+00:00,None,None,Rivne,Ukraine,cadiji4652@brandoza.com,false,"{""abandoned"":1,""bidding"":0,""canceled"":0,""close..."
4,63eb6d339d9c8739fca172c4,2023-02-14 11:14:59.296000+00:00,2023-04-18 09:21:13.644000+00:00,None,None,Trenton,United States,maxym.kobzar@gmail.com,false,"{""abandoned"":3,""bidding"":0,""canceled"":0,""close..."


## 4. Data cleaning and preparation

We clean and standardize column names and data types so that both
sources can be merged reliably and then exported to Meta.


### 4.1 Prepare segmentation dataframe

- Rename relevant columns.
- Keep only the fields we actually need.
- Replace `"Unknown"` with `NaN` for proper missing-value handling.


In [ ]:
# Rename and select only the needed columns from segmentation data
df_seg = df_seg_raw.rename(
    columns={
        "Customer ID": "user_id",
        "Email": "segEmail",
        "Country": "segCountry",
        "First Name": "segFirstName",
        "Last Name": "segLastName",
        "Total Value": "value",
        "Total Score": "rfmScore",
    }
)[
    [
        "user_id",
        "segEmail",
        "segCountry",
        "segFirstName",
        "segLastName",
        "value",
        "rfmScore",
    ]
].copy()

# Normalize user_id as string for reliable joins
df_seg["user_id"] = df_seg["user_id"].astype(str).str.strip()

# Replace literal "Unknown" with NaN for later cleaning
cols_with_unknown = ["segFirstName", "segLastName", "segCountry"]
df_seg[cols_with_unknown] = df_seg[cols_with_unknown].replace("Unknown", np.nan)

df_seg.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83369 entries, 0 to 83368
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   user_id       83369 non-null  object 
 1   segEmail      83369 non-null  object 
 2   segCountry    81738 non-null  object 
 3   segFirstName  47039 non-null  object 
 4   segLastName   42310 non-null  object 
 5   value         83369 non-null  float64
 6   rfmScore      83369 non-null  object 
dtypes: float64(1), object(6)
memory usage: 4.5+ MB


### 4.2 Prepare BigQuery dataframe

- Normalize `user_id` to string.
- Make sure emails and country names are trimmed.


In [ ]:
df_bq = df_bq_raw.copy()

df_bq["user_id"] = df_bq["user_id"].astype(str).str.strip()
df_bq["bgEmail"] = df_bq["bgEmail"].astype(str).str.strip()
df_bq["bgCity"] = df_bq["bgCity"].astype(str).str.strip()
df_bq["bgCountry"] = df_bq["bgCountry"].astype(str).str.strip()

df_bq.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83369 entries, 0 to 83368
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   user_id         83369 non-null  object             
 1   createdAt       83369 non-null  datetime64[us, UTC]
 2   lastActivityAt  83369 non-null  datetime64[us, UTC]
 3   firstName       47049 non-null  object             
 4   lastName        44725 non-null  object             
 5   bgCity          83369 non-null  object             
 6   bgCountry       83369 non-null  object             
 7   bgEmail         83369 non-null  object             
 8   emailConfirmed  83369 non-null  object             
 9   statistic       83369 non-null  object             
dtypes: datetime64[us, UTC](2), object(8)
memory usage: 6.4+ MB


## 5. Merge datasets on `user_id`

We perform an inner join on `user_id` to keep only customers that exist
both in BigQuery and in the segmentation table.


In [ ]:
df_merged = df_bq.merge(
    df_seg,
    on="user_id",
    how="inner",
    suffixes=("", "_seg"),
)

print(df_merged.shape)
df_merged.head()


(83369, 16)


,user_id,createdAt,lastActivityAt,firstName,lastName,bgCity,bgCountry,bgEmail,emailConfirmed,statistic,segEmail,segCountry,segFirstName,segLastName,value,rfmScore
0,63e3d7039d9c8739fca172aa,2023-02-08 17:08:19.930000+00:00,2023-02-24 07:08:53.636000+00:00,None,None,Rivne,Ukraine,solonska.a@gmail.com,false,"{""abandoned"":1,""bidding"":0,""canceled"":0,""close...",solonska.a@gmail.com,Ukraine,NaN,NaN,0.0,Not applicable
1,63e4bab29d9c8739fca172ab,2023-02-09 09:19:46.188000+00:00,2023-02-21 15:35:04.719000+00:00,None,None,Rivne,Ukraine,olya9k@gmail.com,false,"{""abandoned"":2,""bidding"":0,""canceled"":0,""close...",olya9k@gmail.com,Ukraine,NaN,NaN,0.0,Not applicable
2,63e4bcbf9d9c8739fca172ac,2023-02-09 09:28:31.910000+00:00,2023-11-17 11:13:52.138000+00:00,None,None,Kyiv,Ukraine,danielsoloshenko@gmail.com,true,"{""abandoned"":4,""bidding"":0,""canceled"":0,""close...",danielsoloshenko@gmail.com,Ukraine,NaN,NaN,0.0,Not applicable
3,63e4bcce9d9c8739fca172ad,2023-02-09 09:28:46.138000+00:00,2023-08-26 12:44:26.203000+00:00,None,None,Rivne,Ukraine,cadiji4652@brandoza.com,false,"{""abandoned"":1,""bidding"":0,""canceled"":0,""close...",cadiji4652@brandoza.com,Ukraine,NaN,NaN,0.0,Not applicable
4,63eb6d339d9c8739fca172c4,2023-02-14 11:14:59.296000+00:00,2023-04-18 09:21:13.644000+00:00,None,None,Trenton,United States,maxym.kobzar@gmail.com,false,"{""abandoned"":3,""bidding"":0,""canceled"":0,""close...",maxym.kobzar@gmail.com,United States,NaN,NaN,0.0,Not applicable


## 6. Name and country normalization

We will build unified first/last name columns and normalize country
names to ISO-2 codes required by Meta.


### 6.1 Normalize names

We build unified first/last name columns:

- `fn`: use `firstName` from BigQuery if not empty, otherwise `segFirstName`.
- `ln`: use `lastName` from BigQuery if not empty, otherwise `segLastName`.

Missing values are replaced with empty strings, which is safe for Meta.


In [ ]:
# Replace None with NaN for consistent handling
name_cols = ["firstName", "lastName", "segFirstName", "segLastName"]
df_merged[name_cols] = df_merged[name_cols].replace({None: np.nan})

# Build unified first and last name fields
df_merged["fn"] = np.where(
    df_merged["firstName"].astype(str).str.strip() != "",
    df_merged["firstName"].astype(str).str.strip(),
    df_merged["segFirstName"].astype(str).str.strip(),
)

df_merged["ln"] = np.where(
    df_merged["lastName"].astype(str).str.strip() != "",
    df_merged["lastName"].astype(str).str.strip(),
    df_merged["segLastName"].astype(str).str.strip(),
)

# Final cleanup: empty strings instead of NaN for names
df_merged[["fn", "ln"]] = df_merged[["fn", "ln"]].replace({np.nan: ""})

df_merged[["fn", "ln"]].head()


,fn,ln
0,nan,nan
1,nan,nan
2,nan,nan
3,nan,nan
4,nan,nan


### 6.2 Normalize countries to ISO-2 codes

Meta requires country values in ISO 3166-1 alpha-2 format.
We map all country names from our dataset to their ISO-2 codes.
Unknown or missing values are converted to an empty string.


In [ ]:
country_to_iso = {
    "": "",
    "Albania": "AL",
    "Algeria": "DZ",
    "American Samoa": "AS",
    "Andorra": "AD",
    "Angola": "AO",
    "Anguilla": "AI",
    "Antigua and Barbuda": "AG",
    "Argentina": "AR",
    "Armenia": "AM",
    "Australia": "AU",
    "Austria": "AT",
    "Azerbaijan": "AZ",
    "Bahamas": "BS",
    "Bahrain": "BH",
    "Barbados": "BB",
    "Belarus": "BY",
    "Belgium": "BE",
    "Belize": "BZ",
    "Bhutan": "BT",
    "Bolivia": "BO",
    "Bosnia and Herzegovina": "BA",
    "Botswana": "BW",
    "Brazil": "BR",
    "British Virgin Islands": "VG",
    "Brunei": "BN",
    "Bulgaria": "BG",
    "Burundi": "BI",
    "Cabo Verde": "CV",
    "Cambodia": "KH",
    "Cameroon": "CM",
    "Canada": "CA",
    "Chad": "TD",
    "Chile": "CL",
    "China": "CN",
    "Colombia": "CO",
    "Costa Rica": "CR",
    "Croatia": "HR",
    "Curacao": "CW",
    "Cyprus": "CY",
    "Czechia": "CZ",
    "Denmark": "DK",
    "Djibouti": "DJ",
    "Dominica": "DM",
    "Dominican Republic": "DO",
    "Ecuador": "EC",
    "Egypt": "EG",
    "El Salvador": "SV",
    "Estonia": "EE",
    "Eswatini": "SZ",
    "Faroe Islands": "FO",
    "Fiji": "FJ",
    "Finland": "FI",
    "France": "FR",
    "French Polynesia": "PF",
    "Gabon": "GA",
    "Gambia": "GM",
    "Georgia": "GE",
    "Germany": "DE",
    "Ghana": "GH",
    "Gibraltar": "GI",
    "Greece": "GR",
    "Greenland": "GL",
    "Grenada": "GD",
    "Guadeloupe": "GP",
    "Guam": "GU",
    "Guatemala": "GT",
    "Guinea": "GN",
    "Guyana": "GY",
    "Haiti": "HT",
    "Honduras": "HN",
    "Hong Kong": "HK",
    "Hungary": "HU",
    "Iceland": "IS",
    "India": "IN",
    "Indonesia": "ID",
    "Iran": "IR",
    "Iraq": "IQ",
    "Ireland": "IE",
    "Israel": "IL",
    "Italy": "IT",
    "Ivory Coast": "CI",
    "Jamaica": "JM",
    "Japan": "JP",
    "Jordan": "JO",
    "Kazakhstan": "KZ",
    "Kenya": "KE",
    "Kiribati": "KI",
    "Kosovo": "XK",
    "Kuwait": "KW",
    "Kyrgyzstan": "KG",
    "Latvia": "LV",
    "Lebanon": "LB",
    "Lesotho": "LS",
    "Liberia": "LR",
    "Libya": "LY",
    "Lithuania": "LT",
    "Luxembourg": "LU",
    "Macao": "MO",
    "Madagascar": "MG",
    "Malawi": "MW",
    "Malaysia": "MY",
    "Maldives": "MV",
    "Malta": "MT",
    "Mauritania": "MR",
    "Mauritius": "MU",
    "Mexico": "MX",
    "Moldova": "MD",
    "Monaco": "MC",
    "Mongolia": "MN",
    "Montenegro": "ME",
    "Morocco": "MA",
    "Mozambique": "MZ",
    "Myanmar": "MM",
    "Namibia": "NA",
    "Nepal": "NP",
    "Netherlands": "NL",
    "New Zealand": "NZ",
    "Nicaragua": "NI",
    "Nigeria": "NG",
    "North Macedonia": "MK",
    "Northern Mariana Islands": "MP",
    "Norway": "NO",
    "Oman": "OM",
    "Pakistan": "PK",
    "Palestinian Territory": "PS",
    "Panama": "PA",
    "Papua New Guinea": "PG",
    "Peru": "PE",
    "Philippines": "PH",
    "Poland": "PL",
    "Portugal": "PT",
    "Puerto Rico": "PR",
    "Qatar": "QA",
    "Republic of the Congo": "CG",
    "Reunion": "RE",
    "Romania": "RO",
    "Russia": "RU",
    "Rwanda": "RW",
    "Saint Kitts and Nevis": "KN",
    "Saint Lucia": "LC",
    "Saint Vincent and the Grenadines": "VC",
    "Samoa": "WS",
    "Saudi Arabia": "SA",
    "Senegal": "SN",
    "Serbia": "RS",
    "Seychelles": "SC",
    "Sierra Leone": "SL",
    "Singapore": "SG",
    "Sint Maarten": "SX",
    "Slovakia": "SK",
    "Slovenia": "SI",
    "Solomon Islands": "SB",
    "Somalia": "SO",
    "South Africa": "ZA",
    "South Korea": "KR",
    "Spain": "ES",
    "Sri Lanka": "LK",
    "Suriname": "SR",
    "Sweden": "SE",
    "Switzerland": "CH",
    "Taiwan": "TW",
    "Tajikistan": "TJ",
    "Tanzania": "TZ",
    "Thailand": "TH",
    "The Netherlands": "NL",
    "Timor Leste": "TL",
    "Togo": "TG",
    "Tonga": "TO",
    "Tunisia": "TN",
    "Turkey": "TR",
    "Turks and Caicos Islands": "TC",
    "U.S. Virgin Islands": "VI",
    "Uganda": "UG",
    "Ukraine": "UA",
    "United Arab Emirates": "AE",
    "United Kingdom": "GB",
    "United States": "US",
    "Uruguay": "UY",
    "Uzbekistan": "UZ",
    "Vanuatu": "VU",
    "Venezuela": "VE",
    "Vietnam": "VN",
    "Zambia": "ZM",
    "Zimbabwe": "ZW",
}

df_merged["country_iso"] = (
    df_merged["bgCountry"].astype(str).str.strip().map(country_to_iso).fillna("")
)

df_merged[["bgCountry", "country_iso"]].head()


,bgCountry,country_iso
0,Ukraine,UA
1,Ukraine,UA
2,Ukraine,UA
3,Ukraine,UA
4,United States,US


## 7. Value normalization

- `value`: numeric, rounded to 2 decimal places.
- `rfmScore`: numeric; `"Not applicable"` and missing values are set to `0`
  and also rounded to 2 decimal places.


In [ ]:
# Normalize value (total order value)
df_merged["value"] = (
    pd.to_numeric(df_merged["value"], errors="coerce")
    .fillna(0)
    .round(2)
)

# Normalize rfmScore: "Not applicable" -> 0
df_merged["rfmScore"] = df_merged["rfmScore"].replace("Not applicable", 0)

df_merged["rfmScore"] = (
    pd.to_numeric(df_merged["rfmScore"], errors="coerce")
    .fillna(0)
    .round(2)
)

df_merged[["value", "rfmScore"]].describe()


/tmp/ipython-input-332566330.py:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_merged["rfmScore"] = df_merged["rfmScore"].replace("Not applicable", 0)


,value,rfmScore
count,83369.000000,83369.000000
mean,57.890041,0.283247
std,345.336529,1.511866
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,21579.480000,12.000000


## 8. Build export DataFrames for Meta

We prepare two export tables with a common schema:

- `EMAIL`
- `FN`
- `LN`
- `CT` (city)
- `COUNTRY` (ISO-2 code)
- `VALUE` (metric for value-based audiences)

Export variants:

1. `meta_order_value.csv` — VALUE = order value.
2. `meta_rfm_score.csv` — VALUE = RFM score.


In [ ]:
# Base columns shared by both audiences
base_export = pd.DataFrame(
    {
        "EMAIL": df_merged["bgEmail"].astype(str).str.strip(),
        "FN": df_merged["fn"],
        "LN": df_merged["ln"],
        "CT": df_merged["bgCity"].astype(str).str.strip(),
        "COUNTRY": df_merged["country_iso"],
    }
)

# 1) Audience based on total order value
meta_order_value = base_export.copy()
meta_order_value["VALUE"] = df_merged["value"]

# 2) Audience based on RFM score
meta_rfm_score = base_export.copy()
meta_rfm_score["VALUE"] = df_merged["rfmScore"]

# Drop records without email (Meta will ignore them, but we clean them up)
meta_order_value = meta_order_value[meta_order_value["EMAIL"] != ""].reset_index(
    drop=True
)
meta_rfm_score = meta_rfm_score[meta_rfm_score["EMAIL"] != ""].reset_index(drop=True)

meta_order_value.head()


,EMAIL,FN,LN,CT,COUNTRY,VALUE
0,solonska.a@gmail.com,nan,nan,Rivne,UA,0.0
1,olya9k@gmail.com,nan,nan,Rivne,UA,0.0
2,danielsoloshenko@gmail.com,nan,nan,Kyiv,UA,0.0
3,cadiji4652@brandoza.com,nan,nan,Rivne,UA,0.0
4,maxym.kobzar@gmail.com,nan,nan,Trenton,US,0.0


## 9. Export CSV files from Colab

We save the prepared tables as CSV files and download them locally
so they can be uploaded to Meta as customer lists / value-based audiences.


In [ ]:
from google.colab import files

ORDER_VALUE_FILENAME = "meta_order_value.csv"
RFM_SCORE_FILENAME = "meta_rfm_score.csv"

meta_order_value.to_csv(ORDER_VALUE_FILENAME, index=False)
meta_rfm_score.to_csv(RFM_SCORE_FILENAME, index=False)

print("meta_order_value.csv shape:", meta_order_value.shape)
print("meta_rfm_score.csv shape:", meta_rfm_score.shape)

files.download(ORDER_VALUE_FILENAME)
files.download(RFM_SCORE_FILENAME)


meta_order_value.csv shape: (83369, 6)
meta_rfm_score.csv shape: (83369, 6)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>